# Modeling Real-Time Evapotranspiration, Growing Degree Days, and Soil Moisture in the State of Minnesota with Stochastic Indices and Geospatial Prediction 

## 1. Set-Up Necessary Modules and Establish Pipeline for Interpolation, Flask API and SDE Connection

### Import Necessary Modules

In [1]:
import pandas as pd # Provides data structures like DataFrames to efficiently work with structured data.
import arcpy # Python interface for working with GIS data and performing geoprocessing tasks.
from arcpy.sa import * # Sub-module of ArcPy that provides access to the Spatial Analyst tools in ArcGIS. 
import numpy as np # Provides support for efficient numerical operations on multi-dimensional arrays and various mathematical functions.
import arcgis # Python library provided by Esri for working with ArcGIS Online and ArcGIS Enterprise.
import requests # Simplifies the process of making HTTP requests, handling response data, and interacting with web services and APIs.
import os # Allows to perfrom operations related to file and directory handling, environment variables, and more.
import math # Basic mathematical operations, trigonometry, logarithmic calculations, and more.
import sys # provides access to system-specific parameters and functions.


sys.path.append(r"C:\Users\cason\Documents\GIS5572\Final Project")
from utils.interpolation import Pipeline # Allows to intergrate Python file to run with in file directory/Github path

### Assign file directories (File Path, Local GDB, and SDE)

In [2]:
output_gdb = r"C:\Users\cason\Documents\GIS5572\Final Project\Final Project.gdb"
output_directory = r"C:\Users\cason\Documents\GIS5572\Final Project"
sde_connection = r"C:\Users\cason\Documents\GIS5572\Final Project\34.135.163.144.sde"

## 2. Perform QAQC, Extract Data from Sources, and Run Models/Evalation Metrics

### Set Function for getting NDAWN URL for each station, by varible and for URL return the URL

In [3]:
def get_ndawn_url(year, begin_date, end_date):
    stations = "78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=185&station=29&station=30&station=154&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=125&station=176&station=177&station=8&station=180&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110"
    variables = "ddavt&variable=ddtpetp"
    ndawn_url = f"https://ndawn.ndsu.nodak.edu/table.csv?station={stations}&variable={variables}&year={year}&ttype=daily&quick_pick=&begin_date={begin_date}&end_date={end_date}"
    return ndawn_url

### Assign the NDAWN URL to Pandas Dataframe

In [4]:
year = 2023
begin_date = "2023-05-08"
end_date = "2023-05-08"
ndawn_url = get_ndawn_url(year, begin_date, end_date)
df = pd.read_csv(ndawn_url, skiprows=[0, 1, 2, 4])
df.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag,Penman PET,Penman PET Flag
0,Ada,47.321190,-96.514060,910,2023,5,8,52.601,NaN,0.066,NaN
1,Adams,48.499880,-98.075880,1580,2023,5,8,48.028,NaN,0.035,NaN
2,Alamo,48.546520,-103.471860,2157,2023,5,8,52.945,NaN,0.115,NaN
3,Alexander,47.750560,-103.733580,2202,2023,5,8,51.268,NaN,0.153,NaN
4,Alvarado,48.245942,-97.021532,809,2023,5,8,51.260,NaN,0.043,NaN


### Set the boundary/coordinates for Minnesota and display them by these locations in a Dataframe

In [5]:
# Define the latitude and longitude ranges for Minnesota
min_lat = 43.5
max_lat = 49.5
min_lon = -97.5
max_lon = -89.5

# Filter the dataframe to only include locations in Minnesota
mn_df = df[(df['Latitude'] >= min_lat) & (df['Latitude'] <= max_lat) & (df['Longitude'] >= min_lon) & (df['Longitude'] <= max_lon)]

# Display first five rows of updated table
mn_df.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag,Penman PET,Penman PET Flag
0,Ada,47.321190,-96.514060,910,2023,5,8,52.601,NaN,0.066,NaN
4,Alvarado,48.245942,-97.021532,809,2023,5,8,51.260,NaN,0.043,NaN
8,Ayr,47.046390,-97.494810,1215,2023,5,8,52.182,NaN,0.090,NaN
13,Becker,45.343990,-93.850140,942,2023,5,8,59.652,NaN,0.203,NaN
23,Campbell,46.064941,-96.370141,987,2023,5,8,55.146,NaN,0.120,NaN


### Model 1: Evapotranspiration (See Diagram and NDAWN)

In [6]:
# Can use a open source module called pyeto that uses Penman eqaution and datetime module and parameters include:

import pyeto
from datetime import datetime

latitude = 35.5  # Latitude of the location in decimal degrees
altitude = 100  # Altitude of the location in meters
temperature = 25  # Air temperature in degrees Celsius
humidity = 60  # Relative humidity in percentage
wind_speed = 3  # Wind speed in meters per second
solar_radiation = 20  # Solar radiation in MJ/m^2/day
# THEN:
date = datetime(2023, 5, 1)  # Example date (year, month, day)

evapotranspiration = pyeto.penman(latitude, altitude, temperature, humidity, wind_speed, solar_radiation, date)

print(evapotranspiration)


ModuleNotFoundError: No module named 'pyeto'

In [7]:
actual = [1,2,3,4,5]
predict = [1,2.5,3,4.9,4.9]

corr_matrix = numpy.corrcoef(actual, predict)
corr = corr_matrix[0,1]
R_sq = corr**2

print(R_sq)

0.934602946460654


In [8]:
y_actual = [1,2,3,4,5]
y_predicted = [1.6,2.5,2.9,3,4.1]
 
MSE = np.square(np.subtract(y_actual,y_predicted)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.6971370023173351


### Model 2: Calculate Growing Degree Days (GDD)

In [9]:
def calculate_gdd(temp_data, base_temp):
    """
    Calculates growing degree days (GDD) from daily temperature data in Celsius
    
    Parameters:
    temp_data (pandas.Series or float): A pandas series or float containing daily temperature data in Celsius
    base_temp (float): The base temperature in Celsius
    
    Returns:
    float: The total growing degree days
    """
    if isinstance(temp_data, pd.Series):
        # Calculate daily GDD
        daily_gdd = temp_data.apply(lambda x: max(x - base_temp, 0))

        # Sum up the daily GDD to get the total GDD
        total_gdd = daily_gdd.sum()
    else:
        total_gdd = max(temp_data - base_temp, 0)

    return total_gdd

### Assign GDD Equation to Dataframe table

In [10]:
# Define the base temperature for GDD calculation
base_temp = 50

# Apply the calculate_gdd function to each row in the DataFrame
mn_df['GDD'] = mn_df['Avg Temp'].apply(lambda x: calculate_gdd(x, base_temp))

[10]:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
mn_df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag,Penman PET,Penman PET Flag,GDD
0,Ada,47.321190,-96.514060,910,2023,5,8,52.601,NaN,0.066,NaN,2.601
4,Alvarado,48.245942,-97.021532,809,2023,5,8,51.260,NaN,0.043,NaN,1.260
8,Ayr,47.046390,-97.494810,1215,2023,5,8,52.182,NaN,0.090,NaN,2.182
13,Becker,45.343990,-93.850140,942,2023,5,8,59.652,NaN,0.203,NaN,9.652
23,Campbell,46.064941,-96.370141,987,2023,5,8,55.146,NaN,0.120,NaN,5.146
29,Clarissa,46.111545,-94.905826,1304,2023,5,8,53.126,NaN,0.117,NaN,3.126
48,Ekre,46.540050,-97.140940,1053,2023,5,8,53.094,NaN,0.089,NaN,3.094
49,Elbow Lake,45.963080,-95.986820,1207,2023,5,8,56.390,NaN,0.144,NaN,6.390
50,Eldred,47.687690,-96.822210,861,2023,5,8,51.971,NaN,0.051,NaN,1.971
51,Emerado,47.912010,-97.325150,877,2023,5,8,51.638,NaN,0.049,NaN,1.638


### Evalation Metrics R-Sqaured and RSME

In [12]:
actual = [1,2,3,4,5]
predict = [1,2.5,3,4.9,4.9]

corr_matrix = numpy.corrcoef(actual, predict)
corr = corr_matrix[0,1]
R_sq = corr**2

print(R_sq)

0.934602946460654


In [13]:
y_actual = [1,2,3,4,5]
y_predicted = [1.6,2.5,2.9,3,4.1]
 
MSE = np.square(np.subtract(y_actual,y_predicted)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.6971370023173351


### Model 3: Soil and Water Assessment Tool (SWAT)

### Extract Crop/Soil Residue Data

In [14]:
# Crop/Soil Data Extraction
Crop_Link = r'https://gisdata.mn.gov/dataset/env-cov-crop-res'

req_obj = requests.get(Crop_Link)

Crop_Data = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'

output = requests.post(Crop_Data)

slashstuff = output.content

zipp = zipfile.ZipFile(io.BytesIO(slashstuff))

zipp.extractall(r'C:\Users\cason\Documents\GIS5572\Final Project')

NameError: name 'zipfile' is not defined

### Save Data to Local GDB

In [15]:
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="cover_crop_residue_agroecoregion",
    Output_Geodatabase= r"C:\Users\cason\Documents\GIS5572\Final Project\Final Project.gdb"
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset cover_crop_residue_agroecoregion does not exist or is not supported
Failed to execute (FeatureClassToGeodatabase).


### Polygon to Raster Tool (County Level)

In [16]:
arcpy.conversion.PolygonToRaster(
    in_features="cover_crop_residue_counties",
    value_field="covacr2019",
    out_rasterdataset=r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS II\Week 10\Lab02.gdb\cover_crop_residue_counties_PolygonToRaster",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=2300,
    build_rat="BUILD"
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset cover_crop_residue_counties does not exist or is not supported
ERROR 000875: Output Raster Dataset: C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS II\Week 10\Lab02.gdb\cover_crop_residue_counties_PolygonToRaster's workspace is an invalid output workspace.
Failed to execute (PolygonToRaster).


### Raster to Point Tool

In [18]:
arcpy.conversion.RasterToPoint(
    in_raster="cover_crop_residue_counties_PolygonToRaster",
    out_point_features=r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS II\Week 10\Lab02.gdb\RasterT_cover_c1",
    raster_field="Value"
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000865: Input raster: cover_crop_residue_counties_PolygonToRaster does not exist.
WARNING 001000: Field: Field Value does not exist
Failed to execute (RasterToPoint).


In [ ]:
arcpy.ga.ExploratoryInterpolation(
    in_features="RasterT_cover_c1",
    value_field="grid_code",
    out_cv_table=r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS II\Week 10\Lab02.gdb\ExploratoryInterpolation1",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING;EBK;KERNEL_INTERPOLATION",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

### Implement SWAT Model

In [ ]:
import pyswat

# Create a new SWAT project
project = pyswat.Project()

# Set the SWAT executable path
project.swat_exe =  r'C:\Users\cason\Documents\GIS5572\Final Project'

# Set the SWAT project directory
project.directory = r'C:\Users\cason\Documents\GIS5572\Final Project'

# Add input files to the project
project.add_file("climate.dbf", "path_to_climate_file.dbf")
project.add_file("soils.dbf", "path_to_soils_file.dbf")
project.add_file("landuse.dbf", "path_to_landuse_file.dbf")

# Run the SWAT model
project.run_model()


### Evalation Metrics R-Sqaured and RSME

In [ ]:
actual = [1,2,3,4,5]
predict = [1,2.5,3,4.9,4.9]

corr_matrix = numpy.corrcoef(actual, predict)
corr = corr_matrix[0,1]
R_sq = corr**2

print(R_sq)

In [ ]:
y_actual = [1,2,3,4,5]
y_predicted = [1.6,2.5,2.9,3,4.1]
 
MSE = np.square(np.subtract(y_actual,y_predicted)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

### Confusion Matrix Assessment for all Models:

## Save Data to Local Geodatabase

In [ ]:
ndawn_sedf = arcgis.GeoAccessor.from_xy(mn_df, "Longitude", "Latitude")

In [ ]:
ndawn_sedf.spatial.to_featureclass(location=os.path.join(output_gdb, "ndawn_observations_gdd"))
ndawn_sedf.spatial.to_featureclass(location=os.path.join(output_gdb, "ndawn_observations_pet"))

# 3. Run Interpolation and Accuracy Assessment/Ground Truth Control and Save to SDE for Display on ArcGIS Online

### Assign File Paths for Evapotranspiration Data and GDD

In [ ]:
gdd = r"C:\Users\cason\Documents\GIS5572\Final Project\Final Project.gdb\ndawn_observations_gdd"
pet = r"C:\Users\cason\Documents\GIS5572\Final Project\Final Project.gdb\ndawn_observations_pet"

In [ ]:
gdd_pipeline = Pipeline(gdd, output_directory, output_gdb, "GDD")

### Exploratory Kriging Analysis for NDAWN and GDD

In [ ]:
gdd_pipeline.run_exploratory_interpolation()

In [ ]:
gdd_pipeline.display("DATAFRAME") # Data Frame Results

### Store Results of Interpolation to Points and Hexagons for Data Aggregation 

In [ ]:
gdd_pipeline.create_point_accuracy_layer()

In [ ]:
gdd_pipeline.convert_results_to_hex(contours=True)

### Exploratory Kriging Analysis for Evapotranspiration 

In [ ]:
pet_pipeline = Pipeline(pet, output_directory, output_gdb, "penman_pet")

In [ ]:
pet_pipeline.run_exploratory_interpolation()

In [ ]:
pet_pipeline.display("DATAFRAME") # Data Frame Result

### Store Results of Interpolation to Points and Hexagons for Data Aggregation 

In [ ]:
pet_pipeline.create_point_accuracy_layer()

In [ ]:
pet_pipeline.convert_results_to_hex(contours=True)

### Export path to Spatial Data Engine 

In [246]:
gdd_pipeline.export_to_sde(sde_connection, "TESSELLATION")

In [247]:
gdd_pipeline.export_to_sde(sde_connection, "POINT_ACCURACY")

In [248]:
pet_pipeline.export_to_sde(sde_connection, "TESSELLATION")

In [249]:
pet_pipeline.export_to_sde(sde_connection, "POINT_ACCURACY")